## Librerias que utilizo

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
%matplotlib inline

## Importacion de la data y validaciones iniciales

In [2]:
titanic = pd.read_csv("titanic_MD.csv")
titanicG = pd.read_csv("titanic.csv")

print('Titanic Malo shape:', titanic.shape)
print('Titanic Bueno shape:', titanicG.shape)

Titanic Malo shape: (183, 12)
Titanic Bueno shape: (183, 12)


In [3]:
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",?,38.0,1.0,0.0,PC 17599,71.2833,C85,C
1,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1.0,0.0,113803,53.1000,C123,S
2,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0.0,0.0,17463,51.8625,E46,S
3,11,1,3,"Sandstrom, Miss. Marguerite Rut",female,NaN,1.0,NaN,PP 9549,16.7000,G6,S
4,12,1,1,"Bonnell, Miss. Elizabeth",female,58.0,NaN,0.0,113783,26.5500,C103,S


In [4]:
titanic.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,183.000000,183.000000,183.000000,158.000000,180.000000,171.000000,175.000000
mean,455.366120,0.672131,1.191257,35.692532,0.461111,0.461988,78.959191
std,247.052476,0.470725,0.515187,15.640858,0.646122,0.753435,77.026328
min,2.000000,0.000000,1.000000,0.920000,0.000000,0.000000,0.000000
25%,263.500000,0.000000,1.000000,24.000000,0.000000,0.000000,29.700000
50%,457.000000,1.000000,1.000000,35.500000,0.000000,0.000000,56.929200
75%,676.000000,1.000000,1.000000,48.000000,1.000000,1.000000,90.539600
max,890.000000,1.000000,3.000000,80.000000,3.000000,4.000000,512.329200


## 1. Datos faltantes por columna

#### Buscando NAs

In [5]:
titanic.isna().sum()

PassengerId     0
Survived        0
Pclass          0
Name            0
Sex             0
Age            25
SibSp           3
Parch          12
Ticket          0
Fare            8
Cabin           0
Embarked       12
dtype: int64

#### Buscando valores extraños en valores String. De todos, solo Sex mostró valores diferentes fáciles de detectar

In [6]:
cols = []
val = []
for col in titanic.select_dtypes(include='object').columns:
    cols.append(col)
    val.append(titanic[col].str.contains(r'\?').sum())
pd.DataFrame({
    'cols':cols,
    'val':val
})

,cols,val
0,Name,0
1,Sex,51
2,Ticket,0
3,Cabin,0
4,Embarked,0


#### Buscando valores anormales en la data numérica. Age y Fare contienen valores extraños ya que Age no puede tener decimales y Fare no puede tener 0s

In [7]:
np.unique(titanic['Age'])

array([ 0.92,  2.  ,  3.  ,  4.  ,  6.  , 11.  , 14.  , 15.  , 16.  ,
       17.  , 18.  , 19.  , 21.  , 22.  , 23.  , 24.  , 25.  , 27.  ,
       28.  , 29.  , 30.  , 31.  , 32.  , 33.  , 34.  , 35.  , 36.  ,
       36.5 , 37.  , 38.  , 39.  , 40.  , 41.  , 42.  , 43.  , 44.  ,
       45.  , 46.  , 47.  , 48.  , 49.  , 50.  , 51.  , 52.  , 53.  ,
       54.  , 55.  , 56.  , 58.  , 60.  , 61.  , 62.  , 63.  , 64.  ,
       65.  , 70.  , 71.  , 80.  ,   nan])

In [8]:
titanic["Fare"].quantile([.00, .10, .25, .50, .75, .90, 1.00])

0.00      0.00000
0.10     13.31668
0.25     29.70000
0.50     56.92920
0.75     90.53960
0.90    153.46250
1.00    512.32920
Name: Fare, dtype: float64

### Survived, Pclass están bien. Sex, Age, SibSp, Parch, Fare y Embarked tienen datos faltantes detallados previamente.
### Se excluyeron de la exploración NAME, CABIN, PASSENGERID y TICKET, ya que estas cuatro columnas describen a una sola persona, no a un conjunto, por lo que a la hora de predecir si puede sobrevivir o no, estas columnas muy específicas no nos servirían

## 2. Tipo de limpieza de datos por columna

#### Sex (texto): usaré la Moda Sectorizada porque puedo sectorizar por valores que ya tengo para sacar cuál es el sexo que más se repite por sector y asumir que esos faltantes lo son, ya que 51 observaciones si afecta.
#### Age (número): usaré la Media porque el promedio de todos a bordo puede abarcar mejor el rango normal, aunque los valores extremos pueden fallar, los valores centrales de la distribución pueden estar muy cerca de la imputación.
#### SibSp (número): usaré la Moda porque son solo 3 datos faltantes y partiré por el supuesto de que esas 3 personas estaban casadas y tenian el mismo número de hijos por temas culturales de la época.
#### Parch (número): usaré la Media aproximada ya que son 12 datos, no afectará mucho y son solo valores enteros. Además, voy a partir del supuesto de que esas 12 personas tienen el promedio de familiares extra a bordo pero aproximado al valor más cercano.
#### Fare (número): usaré una Regresión Lineal ya que puedo usar la clase y valores faltantes calculados (como acompañantes) para predecir el posible valor de su ticket.
#### Embarked (texto): usaré la Moda porque son pocos datos faltantes (no influye mucho en el modelo) y partiré por el supuesto de que esas 12 personas abordaron en el mismo puerto.

## 3. Cantidad de filas completas

In [9]:
titanic.replace(r'\?', np.nan, regex = True, inplace = True)
titanic.dropna().shape

(100, 12)

In [10]:
# % por cada columna
nas = pd.DataFrame(titanic.isna().sum()/titanic.shape[0], columns = ['%NAs']).reset_index()
nas

,index,%NAs
0,PassengerId,0.000000
1,Survived,0.000000
2,Pclass,0.000000
3,Name,0.000000
4,Sex,0.278689
5,Age,0.136612
6,SibSp,0.016393
7,Parch,0.065574
8,Ticket,0.000000
9,Fare,0.043716


#### Hay 100 filas completas en la tabla, tomando las consideraciones descritas arriba

## 4. Eliminación de NAs

#### Imputacion por media, mediana y moda

In [11]:
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
imp_median = SimpleImputer(missing_values=np.nan, strategy='median')
imp_mode = SimpleImputer(missing_values=np.nan, strategy='most_frequent')

In [13]:
#Sex
comparativa = pd.DataFrame(imp_mode.fit_transform(titanic[['Sex']]), columns = ['Sex_mode'])

In [14]:
#Age
#Media
comparativa['Age_mean'] = imp_mean.fit_transform(titanic[['Age']])
comparativa['Age_mean'] = round(comparativa['Age_mean'], 2)

#Mediana
comparativa['Age_median'] = imp_median.fit_transform(titanic[['Age']])

#Moda
comparativa['Age_mode'] = imp_mode.fit_transform(titanic[['Age']])

In [15]:
#SibSp
#Media
comparativa['SibSp_mean'] = imp_mean.fit_transform(titanic[['SibSp']])
comparativa['SibSp_mean'] = round(comparativa['SibSp_mean'])

#Mediana
comparativa['SibSp_median'] = imp_median.fit_transform(titanic[['SibSp']])
comparativa['SibSp_median'] = round(comparativa['SibSp_median'])

#Moda
comparativa['SibSp_mode'] = imp_mode.fit_transform(titanic[['SibSp']])
comparativa['SibSp_mode'] = round(comparativa['SibSp_mode'])

In [16]:
#Parch
#Media
comparativa['Parch_mean'] = imp_mean.fit_transform(titanic[['Parch']])
comparativa['Parch_mean'] = round(comparativa['Parch_mean'])

#Mediana
comparativa['Parch_median'] = imp_median.fit_transform(titanic[['Parch']])
comparativa['Parch_median'] = round(comparativa['Parch_median'])

#Moda
comparativa['Parch_mode'] = imp_mode.fit_transform(titanic[['Parch']])
comparativa['Parch_mode'] = round(comparativa['Parch_mode'])

In [17]:
#Fare
#Media
comparativa['Fare_mean'] = imp_mean.fit_transform(titanic[['Fare']])
comparativa['Fare_mean'] = round(comparativa['Fare_mean'], 2)

#Mediana
comparativa['Fare_median'] = imp_median.fit_transform(titanic[['Fare']])

#Moda
comparativa['Fare_mode'] = imp_mode.fit_transform(titanic[['Fare']])

In [18]:
#Embarked
comparativa['Embarked_mode'] = imp_mode.fit_transform(titanic[['Embarked']])
comparativa

,Sex_mode,Age_mean,Age_median,Age_mode,SibSp_mean,SibSp_median,SibSp_mode,Parch_mean,Parch_median,Parch_mode,Fare_mean,Fare_median,Fare_mode,Embarked_mode
0,male,38.00,38.0,38.0,1.0,1.0,1.0,0.0,0.0,0.0,71.28,71.2833,71.2833,C
1,female,35.00,35.0,35.0,1.0,1.0,1.0,0.0,0.0,0.0,53.10,53.1000,53.1000,S
2,male,54.00,54.0,54.0,0.0,0.0,0.0,0.0,0.0,0.0,51.86,51.8625,51.8625,S
3,female,35.69,35.5,24.0,1.0,1.0,1.0,0.0,0.0,0.0,16.70,16.7000,16.7000,S
4,female,58.00,58.0,58.0,0.0,0.0,0.0,0.0,0.0,0.0,26.55,26.5500,26.5500,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178,female,47.00,47.0,47.0,1.0,1.0,1.0,1.0,1.0,1.0,78.96,56.9292,26.5500,S
179,male,35.69,35.5,24.0,0.0,0.0,0.0,0.0,0.0,0.0,5.00,5.0000,5.0000,S
180,female,56.00,56.0,56.0,0.0,0.0,0.0,0.0,0.0,0.0,83.16,83.1583,83.1583,S
181,male,19.00,19.0,19.0,0.0,0.0,0.0,0.0,0.0,0.0,30.00,30.0000,30.0000,S


#### Regresión Lineal

In [19]:
titanic_clean = titanic.dropna()
titanic_clean.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1.0,0.0,113803,53.1000,C123,S
2,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0.0,0.0,17463,51.8625,E46,S
5,22,1,2,"Beesley, Mr. Lawrence",male,34.0,0.0,0.0,248698,13.0000,D56,S
9,55,0,1,"Ostby, Mr. Engelhart Cornelius",male,65.0,0.0,1.0,113509,61.9792,B30,C
10,63,0,1,"Harris, Mr. Henry Birkhardt",male,45.0,1.0,0.0,36973,83.4750,C83,S


In [20]:
def regre(x,y): 
    lm = LinearRegression()
    lm.fit(x,y)
    y_hat = lm.predict(x)
    error = (1/2*np.mean(y_hat - y ) **2)
    
    #creación de un data frame
    df_dict = { 'PassengerId':titanic_clean.dropna().PassengerId, 'y_hat': y_hat }
    df = pd.DataFrame(df_dict)
    return(lm)

In [21]:
#Age
age_lm = regre(titanic_clean.dropna().drop(['PassengerId','Name', 'Sex', 'Ticket', 'Cabin', 'Embarked', 'Age'], axis = 1), titanic_clean['Age'].dropna())
titanic_clean['Age_lm'] = age_lm.predict(titanic_clean.drop(['PassengerId','Name', 'Sex', 'Ticket', 'Cabin', 'Embarked', 'Age'], axis = 1))
titanic_clean['Age_lm']

C:\Users\danie\AppData\Local\Temp\ipykernel_15996\3219278990.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  titanic_clean['Age_lm'] = age_lm.predict(titanic_clean.drop(['PassengerId','Name', 'Sex', 'Ticket', 'Cabin', 'Embarked', 'Age'], axis = 1))


1      38.934680
2      45.396153
5      29.657247
9      38.803965
10     48.797313
         ...    
171    31.649758
172    17.238063
173    31.886326
174    28.763002
177    45.427416
Name: Age_lm, Length: 100, dtype: float64

In [22]:
#SibSp
SibSp_lm = regre(titanic_clean.dropna().drop(['PassengerId','Name', 'Sex', 'Ticket', 'Cabin', 'Embarked', 'Age'], axis = 1), titanic_clean['SibSp'].dropna())
titanic_clean['Sibsp_lm'] = SibSp_lm.predict(titanic_clean.drop(['PassengerId','Name', 'Sex', 'Ticket', 'Cabin', 'Embarked', 'Age'], axis = 1))
titanic_clean['Sibsp_lm']

C:\Users\danie\AppData\Local\Temp\ipykernel_15996\1793842924.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  titanic_clean['Sibsp_lm'] = SibSp_lm.predict(titanic_clean.drop(['PassengerId','Name', 'Sex', 'Ticket', 'Cabin', 'Embarked', 'Age'], axis = 1))


1      1.000000e+00
2      4.218847e-15
5     -1.532108e-14
9      1.776357e-15
10     1.000000e+00
           ...     
171    1.000000e+00
172   -2.287059e-14
173    1.000000e+00
174   -1.132427e-14
177    3.996803e-15
Name: Sibsp_lm, Length: 100, dtype: float64

In [23]:
#Parch
Parch_lm = regre(titanic_clean.dropna().drop(['PassengerId','Name', 'Sex', 'Ticket', 'Cabin', 'Embarked', 'Age'], axis = 1), titanic_clean['Parch'].dropna())
titanic_clean['Parch_lm'] = Parch_lm.predict(titanic_clean.drop(['PassengerId','Name', 'Sex', 'Ticket', 'Cabin', 'Embarked', 'Age'], axis = 1))
titanic_clean['Parch_lm']

C:\Users\danie\AppData\Local\Temp\ipykernel_15996\466659140.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  titanic_clean['Parch_lm'] = Parch_lm.predict(titanic_clean.drop(['PassengerId','Name', 'Sex', 'Ticket', 'Cabin', 'Embarked', 'Age'], axis = 1))


1      2.220446e-16
2      1.110223e-15
5      2.442491e-15
9      1.000000e+00
10    -6.661338e-16
           ...     
171    1.000000e+00
172    1.000000e+00
173    1.000000e+00
174    1.000000e+00
177    1.110223e-15
Name: Parch_lm, Length: 100, dtype: float64

In [24]:
#Fare
Fare_lm = regre(titanic_clean.dropna().drop(['PassengerId','Name', 'Sex', 'Ticket', 'Cabin', 'Embarked', 'Age'], axis = 1), titanic_clean['Fare'].dropna())
titanic_clean['Fare_lm'] = Fare_lm.predict(titanic_clean.drop(['PassengerId','Name', 'Sex', 'Ticket', 'Cabin', 'Embarked', 'Age'], axis = 1))
titanic_clean['Fare_lm']

C:\Users\danie\AppData\Local\Temp\ipykernel_15996\12869493.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  titanic_clean['Fare_lm'] = Fare_lm.predict(titanic_clean.drop(['PassengerId','Name', 'Sex', 'Ticket', 'Cabin', 'Embarked', 'Age'], axis = 1))


1      53.1000
2      51.8625
5      13.0000
9      61.9792
10     83.4750
        ...   
171    93.5000
172    12.4750
173    83.1583
174    39.4000
177    50.4958
Name: Fare_lm, Length: 100, dtype: float64

In [25]:
titanic_clean.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age_lm,Sibsp_lm,Parch_lm,Fare_lm
1,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1.0,0.0,113803,53.1000,C123,S,38.934680,1.000000e+00,2.220446e-16,53.1000
2,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0.0,0.0,17463,51.8625,E46,S,45.396153,4.218847e-15,1.110223e-15,51.8625
5,22,1,2,"Beesley, Mr. Lawrence",male,34.0,0.0,0.0,248698,13.0000,D56,S,29.657247,-1.532108e-14,2.442491e-15,13.0000
9,55,0,1,"Ostby, Mr. Engelhart Cornelius",male,65.0,0.0,1.0,113509,61.9792,B30,C,38.803965,1.776357e-15,1.000000e+00,61.9792
10,63,0,1,"Harris, Mr. Henry Birkhardt",male,45.0,1.0,0.0,36973,83.4750,C83,S,48.797313,1.000000e+00,-6.661338e-16,83.4750


In [ ]:
comparativa['Age_lm'] = titanic['Age']
comparativa['SibSp_lm'] = titanic['SibSp']
comparativa['Parch_lm'] = titanic['Parch']
comparativa['Fare_lm'] = titanic['Fare']

comparativa["Age_lm"].fillna(titanic_clean['Age_lm'].mean(), inplace = True)
comparativa["SibSp_lm"].fillna(round(titanic_clean['Sibsp_lm'].mean()), inplace = True)
comparativa["Parch_lm"].fillna(round(titanic_clean['Parch_lm'].mean()), inplace = True)
comparativa["Fare_lm"].fillna(titanic_clean['Fare_lm'].mean(), inplace = True)
comparativa

#### Outliers (método usado: Percentiles)

In [ ]:
perc = titanic.dropna()
perc

In [ ]:
#Edad
pl = np.percentile(perc['Age'], 5)
pu = np.percentile(perc['Age'], 95)
sns.scatterplot(x = perc['PassengerId'], y = perc['Age'])
sns.lineplot(x = perc['PassengerId'], y = pl, color = 'red')
sns.lineplot(x = perc['PassengerId'], y = pu, color = 'green')

In [ ]:
procesador = perc[(perc['Age']>=pl) & (perc['Age']<=pu)]
procesador[['Age']].describe()

In [ ]:
perc['Age_drop'] = procesador['Age']

In [ ]:
perc['Age_pcv'] = np.where(
    perc['Age']<pl,
    pl,
    np.where(
        perc['Age']>pu,
        pu,
        perc['Age']
    )
)

In [ ]:
perc[['Age', 'Age_pcv', 'Age_drop']].describe()

In [ ]:
comparativa['Age_out'] = perc['Age_drop']
comparativa